# Stock NeurIPS2018 Part 3. Backtest
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the third and last part of the NeurIPS2018 series, introducing how to use use the agents we trained to do backtest, and compare with baselines such as Mean Variance Optimization and DJIA index.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR

import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parents[1]))
from finai_contest.env_stock_trading.env_stock_trading_finrlmeta import StockTradingEnv_FinRLMeta
from finai_contest.env_stock_trading.env_stock_trading_gym_anytrading import StockTradingEnv_gym_anytrading
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

# Part 2. Backtesting

To backtest the agents, upload trade_data.csv in the same directory of this notebook. For Colab users, just upload trade_data.csv to the default directory.

In [ ]:
train = pd.read_csv('./data/train_data.csv')
trade = pd.read_csv('./data/trade_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']
trade_aapl = trade[trade["tic"] == "AAPL"]


Then, upload the trained agent to the same directory, and set the corresponding variable to True.

In [ ]:
if_using_a2c = False
if_using_ddpg = False
if_using_ppo = True
if_using_td3 = False
if_using_sac = False

Load the agents

In [ ]:
# env_used = "finrl"
env_used = "gym_anytrading"

In [ ]:
trained_a2c = A2C.load(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ddpg = DDPG.load(TRAINED_MODEL_DIR + "/agent_ddpg") if if_using_ddpg else None
trained_ppo = PPO.load(TRAINED_MODEL_DIR + "/agent_ppo_"+env_used) if if_using_ppo else None
trained_td3 = TD3.load(TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None
trained_sac = SAC.load(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
stock_dimension = len(trade_aapl.tic.unique())
if env_used == "finrl":
    state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
    buy_cost_list = sell_cost_list = [0.001] * stock_dimension
elif env_used == "gym_anytrading":
    state_space = 1 + 3 * stock_dimension
    buy_cost_list = [0.005] * stock_dimension
    sell_cost_list = [0.01] * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [ ]:
num_stock_shares = [0] * stock_dimension

if env_used =="finrl":
    env_kwargs = {
        "hmax": 100,
        "initial_amount": 1000000,
        "num_stock_shares": num_stock_shares,
        "buy_cost_pct": buy_cost_list,
        "sell_cost_pct": sell_cost_list,
        "state_space": state_space,
        "stock_dim": stock_dimension,
        "tech_indicator_list": INDICATORS,
        "action_space": stock_dimension,
        "reward_scaling": 1e-4
    }


    e_trade_gym = StockTradingEnv_FinRLMeta(df = trade_aapl, **env_kwargs)

elif env_used =="gym_anytrading":
    env_kwargs = {
    "hmax": np.inf,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": 2*stock_dimension,
    "reward_scaling": 1e-4,
    "window_size": 30
    }

    e_trade_gym = StockTradingEnv_gym_anytrading(df = trade_aapl, **env_kwargs)

In [ ]:
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
# trade_aapl = trade[trade["tic"] == "AAPL"]
# trade_gym_anytrade = pd.DataFrame()
# trade_gym_anytrade['Time'] = pd.to_datetime(trade_aapl['date'])  # Convert to datetime
# trade_gym_anytrade['Open'] = trade_aapl['open']
# trade_gym_anytrade['High'] = trade_aapl['high']
# trade_gym_anytrade['Low'] = trade_aapl['low']
# trade_gym_anytrade['Close'] = trade_aapl['close']
# trade_gym_anytrade['Volume'] = trade_aapl['volume']
# trade_aapl


In [ ]:
# import os
# import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

# import gymnasium as gym
# import gym_anytrading

# e_trade_gym = gym.make(
#     'stocks-v0',
#     df=trade_gym_anytrade,
#     window_size=30,
#     frame_bound=(30, len(trade_gym_anytrade))
# )

# from stable_baselines3.common.vec_env import DummyVecEnv

# def get_sb_env(self):
#     e = DummyVecEnv([lambda: self])
#     obs = e.reset()
#     return e, obs


# def save_asset_memory(self):
#     if "total_profit" not in self.history:
#         print("Warning: 'total_profit' not found in history. Returning empty DataFrame.")
#         return pd.DataFrame({"date": [], "account_value": []})
#     dates = self.df['Time'][self._start_tick:self._current_tick].dt.strftime('%Y-%m-%d')
#     profits = self.history["total_profit"]

#     assert len(dates) == len(profits), f"Length mismatch: {len(dates)} dates vs {len(profits)} profits"

#     df_account_value = pd.DataFrame({
#         "date": dates,
#         "account_value": profits
#     })
#     return df_account_value


# def save_action_memory(self):
#     if "total_profit" not in self.history:
#         print("Warning: 'total_profit' not found in history. Returning empty DataFrame.")
#         return pd.DataFrame({"date": [], "account_value": []})
#     dates = self.df['Time'][self._start_tick:self._current_tick].dt.strftime('%Y-%m-%d')
#     actions = self.history["position"]

#     assert len(dates) == len(actions), f"Length mismatch: {len(dates)} dates vs {len(actions)} profits"

#     df_account_value = pd.DataFrame({
#         "date": dates,
#         "action": actions
#     })
#     return df_account_value

# # Patch the method
# e_trade_gym = e_trade_gym.env.env
# e_trade_gym.get_sb_env = get_sb_env.__get__(e_trade_gym)
# e_trade_gym.save_asset_memory = save_asset_memory.__get__(e_trade_gym)
# e_trade_gym.save_action_memory = save_action_memory.__get__(e_trade_gym)
# e_trade_gym.get_sb_env

In [ ]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym) if if_using_a2c else (None, None)

In [ ]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym) if if_using_ddpg else (None, None)

In [ ]:
df_account_value_ppo_list = []
df_actions_ppo_list = []
for i in range(1):
    df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
        model=trained_ppo, 
        environment = e_trade_gym, deterministic=False) if if_using_ppo else (None, None)
    df_account_value_ppo_list.append(df_account_value_ppo)
    df_actions_ppo_list.append(df_actions_ppo)
    # print(df_account_value_ppo_list)

df_account_value_all = pd.concat([df.set_index("date")["account_value"] for df in df_account_value_ppo_list],axis=1)
df_account_value_all.columns = [f"run_{i+1}" for i in range(len(df_account_value_ppo_list))]
df_account_value_all['mean'] = df_account_value_all.mean(axis=1)
df_account_value_all['std'] = df_account_value_all.std(axis=1)



In [ ]:
df_account_value_ppo = df_account_value_all[['mean', 'std']].reset_index()
df_account_value_ppo.columns = ['date', 'account_value', 'std']

In [ ]:
import os
os.makedirs("./results_csv", exist_ok=True)
df_account_value_ppo.to_csv("./results_csv/account_value_ppo_"+env_used+".csv", index=False)
print(df_account_value_ppo)

In [ ]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [ ]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym) if if_using_sac else (None, None)

In [ ]:
def _calculate_sharpe_ratio(total_profits):
    total_profits = np.array(total_profits)
    
    # Calculate daily returns (percentage change)
    daily_returns = np.diff(total_profits) / total_profits[:-1]
    
    if daily_returns.std() == 0 or len(daily_returns) < 2:
        return 0.0
    
    sharpe = (255 ** 0.5) * daily_returns.mean() / daily_returns.std()
    return sharpe

print("Sharpe Ratio:",_calculate_sharpe_ratio(df_account_value_ppo["account_value"]))

In [ ]:
import matplotlib.pyplot as plt

# Ensure 'date' is in datetime format (optional, for better x-axis formatting)
df_account_value_ppo['date'] = pd.to_datetime(df_account_value_ppo['date'])

# Plot
plt.figure(figsize=(12, 6))
plt.plot(df_account_value_ppo['date'], df_account_value_ppo['account_value'], label='Account Value (PPO)', linewidth=2)
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.title('PPO Trading Strategy: Account Value Over Time')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Part 3: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [ ]:
def process_df_for_mvo(df):
  return df.pivot(index="date", columns="tic", values="close")

### Helper functions for mean returns and variance-covariance matrix

In [ ]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai
# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns): 
  import numpy as np 
  StockReturn = np.zeros([Rows-1, Columns]) 
  for j in range(Columns):        # j: Assets 
    for i in range(Rows-1):     # i: Daily Prices 
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100 
      
  return StockReturn

### Calculate the weights for mean-variance

In [ ]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

In [ ]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
 
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

### Use PyPortfolioOpt

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(len(cleaned_weights_mean))])
mvo_weights

In [ ]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

In [ ]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
MVO_result

# Part 4: DJIA index

Add DJIA index as a baseline to compare with.

In [ ]:
# TRAIN_START_DATE = '2009-01-01'
# TRAIN_END_DATE = '2020-07-01'
# TRADE_START_DATE = '2020-07-01'
# TRADE_END_DATE = '2021-10-29'
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2023-01-01'
TRADE_START_DATE = '2023-01-01'
TRADE_END_DATE = '2024-12-31'

In [ ]:
df_dji = YahooDownloader(
    start_date=TRADE_START_DATE, end_date=TRADE_END_DATE, ticker_list=["^DJI"]
).fetch_data()

In [ ]:
df_dji = df_dji[["date", "close"]]
fst_day = df_dji["close"][0]
dji = pd.merge(
    df_dji["date"],
    df_dji["close"].div(fst_day).mul(1000000),
    how="outer",
    left_index=True,
    right_index=True,
).set_index("date")

# Part 5: Equally Weighted Strategy


In [ ]:
def process_df(df):
  return df.pivot(index="date", columns="tic", values="close")

In [ ]:
StockData = process_df(train)
TradeData = process_df(trade)

TradeData.to_numpy()

In [ ]:
trade_data = TradeData.to_numpy()
T, N = trade_data.shape

# Initialize portfolio
portfolio_value = [1000000]
weights = np.ones(N) / N  # equal weights

for t in range(1, T):
    # Previous prices and today's prices
    prev_prices = trade_data[t - 1]
    curr_prices = trade_data[t]

    # How many shares of each asset we held yesterday
    shares = (portfolio_value[-1] * weights) / prev_prices

    # Today's value = shares * today's price
    new_value = np.sum(shares * curr_prices)

    portfolio_value.append(new_value)

TradeData.index = pd.to_datetime(TradeData.index)

# Step 2: Convert your portfolio_value into a DataFrame
EWS_result = pd.DataFrame(
    portfolio_value, 
    index=TradeData.index,   # align with dates
    columns=["Equal Weight"] # name the strategy
)
EWS_result

<a id='4'></a>
# Part 6: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
df_result_a2c = (
    df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
    if if_using_a2c
    else None
)
df_result_ddpg = (
    df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
    if if_using_ddpg
    else None
)
df_result_ppo = (
    df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
    if if_using_ppo
    else None
)
df_result_td3 = (
    df_account_value_td3.set_index(df_account_value_td3.columns[0])
    if if_using_td3
    else None
)
df_result_sac = (
    df_account_value_sac.set_index(df_account_value_sac.columns[0])
    if if_using_sac
    else None
)
df_result_ppo.index = pd.to_datetime(df_result_ppo.index)
MVO_result.index = pd.to_datetime(MVO_result.index)
dji.index = pd.to_datetime(dji.index)
print(df_result_ppo["account_value"])
print(MVO_result["Mean Var"])
print(df_result_ppo["account_value"].apply(type).unique())
print(MVO_result["Mean Var"].apply(type).unique())
print(dji["close"].apply(type).unique())
result = pd.DataFrame(
    {
        "a2c": df_result_a2c["account_value"] if if_using_a2c else None,
        "ddpg": df_result_ddpg["account_value"] if if_using_ddpg else None,
        "ppo": df_result_ppo["account_value"] if if_using_ppo else None,
        "td3": df_result_td3["account_value"] if if_using_td3 else None,
        "sac": df_result_sac["account_value"] if if_using_sac else None,
        "mvo": MVO_result["Mean Var"],
        "dji": dji["close"],
        "ews": EWS_result["Equal Weight"]
    }
)

In [ ]:
import numpy as np
import pandas as pd

trading_days_per_year = 252

# === Utility Functions ===
def compute_annualized_return(v0, vT, T):
    return (1+((vT - v0)/ v0))** (365 / T) - 1

def compute_sharpe_ratio(daily_returns):
    if daily_returns.std() == 0 or len(daily_returns) < 2:
        return 0.0
    return daily_returns.mean() / daily_returns.std() * np.sqrt(trading_days_per_year)

def compute_annualized_volatility(daily_returns):
    return daily_returns.std() * np.sqrt(trading_days_per_year)

def compute_max_drawdown(daily_returns):
    r = np.asarray(daily_returns, dtype=float)
    if r.size == 0:
        return 0.0
    r = r[~np.isnan(r)]
    if r.size == 0:
        return 0.0
    equity = np.cumprod(1.0 + r)
    peaks = np.maximum.accumulate(equity)
    drawdowns = equity / peaks - 1.0  # ≤ 0
    return float(np.min(drawdowns) * 100.0)
# === Initialize Metrics Dictionary ===
metrics = {}

# === Process Baselines ===
for strategy in result.columns.drop('ppo'):
    series = result[strategy].dropna()
    if len(series) < 2:
        continue

    daily_returns = series.pct_change().dropna()
    final_value = series.iloc[-1]
    v0 = series.iloc[0]
    T = len(series)
    print("baseline trade",T)
    print(strategy,v0,final_value)
    annual_return = compute_annualized_return(v0, final_value, T)
    volatility = compute_annualized_volatility(daily_returns)
    sharpe = compute_sharpe_ratio(daily_returns)
    max_drawdown = compute_max_drawdown(daily_returns)
    metrics[strategy] = {
        "Final Value": final_value,
        "Annualized Return": annual_return,
        "Annualized Volatility": volatility,
        "Annualized StdErr": np.nan,
        "Sharpe Ratio": sharpe,
        "Maximum Drawdown":max_drawdown,
    }

# === Process PPO Multi-run ===
ppo_runs = [df_account_value_all[f'run_{i+1}'] for i in range(1)]
ppo_final_values = []
ppo_annual_returns = []
ppo_sharpe_ratios = []
ppo_volatilities = []
ppo_max_drawdown = []
for series in ppo_runs:
    series = series.dropna()
    v0 = series.iloc[0]
    vT = series.iloc[-1]
    T = len(series)
    print("agent trade",T)

    daily_returns = series.pct_change().dropna()

    ppo_final_values.append(vT)
    ppo_annual_returns.append(compute_annualized_return(v0, vT, T))
    ppo_sharpe_ratios.append(compute_sharpe_ratio(daily_returns))
    ppo_volatilities.append(daily_returns.std() * np.sqrt(252)) 
    ppo_max_drawdown.append(compute_max_drawdown(daily_returns))
# Convert to arrays
ppo_annual_returns = np.array(ppo_annual_returns)
ppo_sharpe_ratios = np.array(ppo_sharpe_ratios)
ppo_final_values = np.array(ppo_final_values)
ppo_volatilities = np.array(ppo_volatilities)
ppo_max_drawdown = np.array(ppo_max_drawdown)

# Compute stats
mean_volatility = ppo_volatilities.mean()
std_volatility = ppo_volatilities.std(ddof=1)

mean_final_value = ppo_final_values.mean()
std_final_value = ppo_final_values.std(ddof=1)
mean_annual_return = ppo_annual_returns.mean()
std_annual_return = ppo_annual_returns.std(ddof=1)
stderr_annual_return = std_annual_return / np.sqrt(len(ppo_annual_returns))
mean_sharpe = ppo_sharpe_ratios.mean()
std_sharpe = ppo_sharpe_ratios.std(ddof=1)
mean_max_drawdown = ppo_max_drawdown.mean()

# Save PPO to metrics
metrics['ppo'] = {
    "Final Value": f"{mean_final_value:.2f}",
    "Annualized Return": f"{mean_annual_return:.4f}",
    "Annualized Volatility": f"{mean_volatility:.4f}",
    "Annualized StdErr": f"{stderr_annual_return:.4f}",
    "Sharpe Ratio": f"{mean_sharpe:.4f}",
    "Maximum Drawdown": f"{mean_max_drawdown:.4f}"

}


# === Final DataFrame ===
df_metrics = pd.DataFrame(metrics)



In [ ]:
df_metrics

In [ ]:
result

Now, everything is ready, we can plot the backtest result.

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
result.plot()